In [1]:
# update command line tool -> 
# (1) csv save (with more columns)
# (2) csv to model file update? (new script)
# (3) augment annotation?
import collections
import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import annotation_maker as am
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
E_COLI_PATH = '/Users/woosubshin/Desktop/AutomateAnnotation/AMAS/AMAS/../tests/e_coli_core.xml'
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')

In [22]:
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)

In [23]:
print(recom.getReactionIDs())

['ODC', 'SAMdc', 'SSAT_for_S', 'SSAT_for_D', 'PAO_for_aD', 'PAO_for_aS', 'SpdS', 'SpmS', 'MAT', 'VCoA', 'VacCoA', 'P_efflux', 'aD_efflux']


In [71]:
rec_upd = pd.read_csv('species_rec.csv')
chosen = rec_upd[(rec_upd['UPDATE ANNOTATION']=='add') |\
                 (rec_upd['UPDATE ANNOTATION']=='delete')]
chosen

AttributeError: 'Series' object has no attribute 'lower'

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.0,1,delete
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,add
2,BIOMD0000000190.xml,species,A,S-adenosylmethioninamine,metaid_0000037,CHEBI:15625,S-adenosylmethioninamine,1.0,1,delete
5,BIOMD0000000190.xml,species,S,Spermine,metaid_0000039,CHEBI:15746,spermine,1.0,1,delete
6,BIOMD0000000190.xml,species,S,Spermine,metaid_0000039,CHEBI:45725,spermine(4+),1.0,0,add
23,BIOMD0000000190.xml,species,CoA,CoA,metaid_0000046,CHEBI:57287,coenzyme A(4-),1.0,0,add


In [16]:
# case 1. add elements, there is no existing annotations
# case 2. add elements, there is existing annotations
# case 3. delete elements, there is no existing annotations
# case 4. delete elements, there is existing annotations 

In [17]:
one_type = 'species'
maker = am.AnnotationMaker(one_type)
df_type = chosen[chosen['type']==one_type]
uids = list(np.unique(df_type['id']))
meta_ids = {val:list(df_type[df_type['id']==val]['meta id'])[0] for val in uids}

In [53]:
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction}

In [56]:
ELEMENT_FUNC[one_type](one_id).getAnnotationString()

'<annotation>\n  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">\n    <rdf:Description rdf:about="#metaid_0000036">\n      <bqbiol:is>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>\n        </rdf:Bag>\n      </bqbiol:is>\n    </rdf:Description>\n  </rdf:RDF>\n</annotation>'

In [46]:
model = recom.sbml_document.getModel()
maker = am.AnnotationMaker(one_type)

for one_id in uids:
  orig_str = ELEMENT_FUNC[one_type](one_id).getAnnotationString()
  df_id = df_type[df_type['id']==one_id]
  adds = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='add'].loc[:, 'annotation'])
  dels = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='delete'].loc[:, 'annotation'])
  deled = maker.deleteAnnotation(dels, orig_str)
  added = maker.addAnnotation(adds, deled, meta_ids[one_id])
  ELEMENT_FUNC[one_type](one_id).setAnnotation(added)

In [47]:
print(np.unique(df_type.id))

['A' 'CoA' 'S' 'SAM']


In [52]:
print(model.getSpecies('CoA').getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000046">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15346"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00010"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:57287"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [59]:
print(recom.sbml_document.getModel().model.getSpecies('CoA').getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000046">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15346"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00010"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:57287"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [67]:
reader = libsbml.SBMLReader()
document = reader.readSBML(BIOMD_190_PATH)
model = document.getModel()
print(model.getSpecies('SAM').getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [66]:
print(recom.sbml_document.getModel().model.getSpecies('SAM').getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [63]:
spec.compartment

'cytosol'

In [64]:
reac = recom.sbml_document.getModel().model.getSpecies(0)

In [65]:
reac.compartment

'cytosol'

In [68]:
SUPPL_FIGURE_ANNOTATION_UPDATE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/writeup/AMAS/Suppl_figures/updating_annotation'

In [70]:
sam_df = rec_upd[rec_upd['id']=='SAM']
sam_df.to_csv(os.path.join(SUPPL_FIGURE_ANNOTATION_UPDATE_DIR, 'sam_df.csv'))